In [44]:
from sklearn.metrics import plot_confusion_matrix, roc_auc_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from service.csv_parser import parse_package
import service.coordinate_service as service

countyData = parse_package('../teir_adatok_2016/megye/')
townData = parse_package('../teir_adatok_2016/telepules/')

/home/lvbubi/PycharmProjects/Accomodation/backend/service/csv_parser.py:24: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dataframe.name = column_names[['id', 'title']]


In [45]:
import matplotlib.pyplot as plt
population = countyData[5]
munkanelkuli = countyData[0]
print(population.name)
print(munkanelkuli.name)

munkanelkuli_ferfi = munkanelkuli[['A', 'B', 'C', 'D']]
munkanelkuli_no = munkanelkuli[['E', 'F', 'G', 'H']]

munkanelkuli_ferfi_prob = munkanelkuli_ferfi.div(population['SUM(A)'], axis=0)
munkanelkuli_ferfi_prob.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)
#plt.matshow(munkanelkuli_ferfi_prob.corr(method='pearson'), cmap='coolwarm')
#plt.show()

                                                   id title
21  Állandó népesség, férfiak összesen (dec. 31.),...     A
22  Állandó népesség, nők összesen (dec. 31.), 201...     B
                                                   id title
21  Teljes munkaidőben foglalkoztatott férfi, 15-2...     A
22  Teljes munkaidőben foglalkoztatott férfi, 15-6...     B
23  Teljes munkaidőben foglalkoztatott férfi, 25-5...     C
24  Teljes munkaidőben foglalkoztatott férfi, 55-6...     D
25  Teljes munkaidőben foglalkoztatott nő, 15-24 é...     E
26  Teljes munkaidőben foglalkoztatott nő, 15-64 é...     F
27  Teljes munkaidőben foglalkoztatott nő, 25-54 é...     G
28  Teljes munkaidőben foglalkoztatott nő, 55-64 é...     H


,A,B,C,D
A,1.0,-0.076,-0.21,-0.13
B,-0.076,1.0,0.97,0.71
C,-0.21,0.97,1.0,0.57
D,-0.13,0.71,0.57,1.0


In [46]:
print(townData[3].name)
print(townData[5].name)
male_population = townData[3]['A']
femaile_population = townData[3]['B']
full_town_population = male_population + femaile_population
print(full_town_population)

                                                     id title
3156  Állandó népesség, férfiak összesen (dec. 31.),...     A
3157  Állandó népesség, nők összesen (dec. 31.), 201...     B
                                                     id title
3156  Nyilvántartott álláskeresők: 0-18 éves, 2016(T...     A
3157  Nyilvántartott álláskeresők: 19-20 éves, 2016(...     B
3158  Nyilvántartott álláskeresők: 21-25 éves, 2016(...     C
3159  Nyilvántartott álláskeresők: 26-30 éves, 2016(...     D
3160  Nyilvántartott álláskeresők: 31-35 éves, 2016(...     E
3161  Nyilvántartott álláskeresők: 36-40 éves, 2016(...     F
3162  Nyilvántartott álláskeresők: 41-45 éves, 2016(...     G
3163  Nyilvántartott álláskeresők: 46-50 éves, 2016(...     H
3164  Nyilvántartott álláskeresők: 51-55 éves, 2016(...     I
3165  Nyilvántartott álláskeresők: 51-60 éves, 2016(...     J
3166  Nyilvántartott álláskeresők: 61-X éves, 2016(T...     K
0       4635.0
1       4297.0
2        697.0
3       2449.0
4         

In [47]:
town_mn = service.add_coordinates_to_town(townData[5])
#town_mn = town_mn[town_mn['county'].str.contains('Borsod') == False]
town_mn = town_mn[::10]
town_mn.fillna(0, inplace=True)

In [48]:
X = town_mn[['A', 'B', 'C', 'D']]
X = X.div(full_town_population, axis=0)
#X.fillna(0, inplace=True)

X = X.values
y = town_mn['county'].values.reshape(-1,1)

In [49]:
# Tréning és teszt adatok létrehozása 2/3 és 1/3 vágással
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ValueError: Found input variables with inconsistent numbers of samples: [3155, 316]

In [ ]:
# 4. SGD
classifier_5 = MLPClassifier(
                    hidden_layer_sizes=(4,),
                    activation='tanh',
                    solver='sgd',
                    max_iter=1000,
                    alpha=0.005,
                    learning_rate='adaptive',
                    random_state=0)

classifier_5.fit(X_train, y_train.ravel())
y_pred_clf_5 = classifier_5.predict(X_test)

plot_confusion_matrix(classifier_5, X_test, y_test, cmap=plt.cm.Blues)
plt.show()

In [ ]:
print('roc auc score: '+str(roc_auc_score(y_true=y_test, y_score=y_pred_clf_5)))
print('f1 score: '+ str(f1_score(y_true=y_test, y_pred=y_pred_clf_5)))
print('Accuracy score: '+ str(classifier_5.score(X_test,y_test)))
print(classification_report(y_true=y_test, y_pred=y_pred_clf_5))

#results.append({'name': 'SGD+TANH(6)', 'accuracy_score': classifier_5.score(X_test,y_test) ,'f1-score': f1_score(y_true=y_test, y_pred=y_pred_clf_5), 'roc_auc_score': roc_auc_score(y_true=y_test, y_score=y_pred_clf_5), 'solver': 'sgd', 'activation': 'tanh'})